# Exploring real world survey contexts
This notebook explores ways of specifying "contexts" in which an [EDSL](https://docs.expectedparrot.com/) survey is administered to AI agents in order to investigate potential impacts to simulated responses. We show how to do this in three different ways by modifying  our `Question` texts and `Agent` traits in order to reflect various hypothetical real world contexts.

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

Thank you to [Sophia Kazinnik](https://sites.google.com/view/skazinnik) for this idea and helpful suggestions!

## Importing the tools

In [1]:
from edsl import QuestionLinearScale, Scenario, ScenarioList, Agent, AgentList, Survey, Model

## Selecting a model
See a list of available models at the [model pricing and performance page](https://www.expectedparrot.com/getting-started/coop-pricing).

In [2]:
# Model.available()

Here we select a model to use:

In [3]:
m = Model("gpt-4o")
m

,key,value
0,model,gpt-4o
1,parameters:temperature,0.500000
2,parameters:max_tokens,1000
3,parameters:top_p,1
4,parameters:frequency_penalty,0
5,parameters:presence_penalty,0
6,parameters:logprobs,False
7,parameters:top_logprobs,3
8,inference_service,openai


## Creating some contexts for our survey
Here we identify some example contexts to use in administering our survey:

In [4]:
respondent_ages = [
    "You are a teenager (13-19 years old).",
    "You are college age (20-24 years old).",
    "You are a young adult (25-39 years old).",
    "You are middle-aged (40-59 years old).",
    "You are a senior citizen (60 or more years old).",
]

survey_contexts = [
    "",
    "You are answering an online survey.",
    "You are being interviewed by a researcher.",
    "You are participating in a focus group of peers.",
    "You are participating in a focus group of people of all ages and backgrounds.",
]

## Adding contexts to question texts
We can apply contexts by creating versions of our questions where each context is inserted directly into the question texts as a `Scenario` of the question:

In [5]:
q_exercise = QuestionLinearScale(
    question_name="exercise",
    question_text="How many times do you typically exercise each week? ({{ scenario.age }} {{ scenario.context }})",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7],
)

q_dessert = QuestionLinearScale(
    question_name="dessert",
    question_text="How many times do you typically eat dessert each week?  ({{ scenario.age }} {{ scenario.context }})",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7],
)

survey = Survey(questions = [q_exercise, q_dessert])

In [6]:
scenarios = ScenarioList(
    Scenario({"age": a, "context": c}) for a in respondent_ages for c in survey_contexts
)

In [7]:
results = survey.by(scenarios).by(m).run()

Job UUID,463f67d7-52d2-47f2-b228-cfbb2e4e3af8
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/463f67d7-52d2-47f2-b228-cfbb2e4e3af8
Exceptions Report URL,None
Results UUID,c48f5437-feca-433f-9ea5-fee5ea831216
Results URL,https://www.expectedparrot.com/content/c48f5437-feca-433f-9ea5-fee5ea831216


In [8]:
results.select("age", "context", "exercise", "dessert")

,scenario.age,scenario.context,answer.exercise,answer.dessert
0,You are a teenager (13-19 years old).,nan,3,3
1,You are a teenager (13-19 years old).,You are answering an online survey.,4,3
2,You are a teenager (13-19 years old).,You are being interviewed by a researcher.,4,3
3,You are a teenager (13-19 years old).,You are participating in a focus group of peers.,3,3
4,You are a teenager (13-19 years old).,You are participating in a focus group of people of all ages and backgrounds.,4,3
5,You are college age (20-24 years old).,nan,3,3
6,You are college age (20-24 years old).,You are answering an online survey.,3,3
7,You are college age (20-24 years old).,You are being interviewed by a researcher.,3,3
8,You are college age (20-24 years old).,You are participating in a focus group of peers.,3,3
9,You are college age (20-24 years old).,You are participating in a focus group of people of all ages and backgrounds.,4,3


## Adding context to agent traits
Another method is to specify agent traits and survey contexts via `Agent` traits instead of the question texts:

In [9]:
q_exercise = QuestionLinearScale(
    question_name="exercise",
    question_text="How many times do you exercise each week?",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7],
)

q_dessert = QuestionLinearScale(
    question_name="dessert",
    question_text="How many times do you eat dessert each week?",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7],
)

survey = Survey([q_exercise, q_dessert])

In [10]:
agents = AgentList(
    Agent(traits={"age": a, "context": c})
    for a in respondent_ages
    for c in survey_contexts
)

In [11]:
results = survey.by(agents).by(m).run()

Job UUID,2ff45f86-5569-41a1-86ef-ad598ffc7126
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/2ff45f86-5569-41a1-86ef-ad598ffc7126
Exceptions Report URL,None
Results UUID,9ddde0c3-125e-4326-84e8-25405c95d381
Results URL,https://www.expectedparrot.com/content/9ddde0c3-125e-4326-84e8-25405c95d381


In [12]:
results.select("age", "context", "exercise", "dessert")

,agent.age,agent.context,answer.exercise,answer.dessert
0,You are a teenager (13-19 years old).,nan,3,3
1,You are a teenager (13-19 years old).,You are answering an online survey.,3,3
2,You are a teenager (13-19 years old).,You are being interviewed by a researcher.,3,3
3,You are a teenager (13-19 years old).,You are participating in a focus group of peers.,3,3
4,You are a teenager (13-19 years old).,You are participating in a focus group of people of all ages and backgrounds.,3,3
5,You are college age (20-24 years old).,nan,3,3
6,You are college age (20-24 years old).,You are answering an online survey.,3,3
7,You are college age (20-24 years old).,You are being interviewed by a researcher.,3,3
8,You are college age (20-24 years old).,You are participating in a focus group of peers.,3,3
9,You are college age (20-24 years old).,You are participating in a focus group of people of all ages and backgrounds.,4,3


## Adding contexts to question texts and agent traits
Here we use both `Agent` traits and `Scenario` contexts in the question texts:

In [13]:
q_exercise = QuestionLinearScale(
    question_name="exercise",
    question_text="How many times do you exercise each week? ({{ scenario.context }})",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7],
)

q_dessert = QuestionLinearScale(
    question_name="dessert",
    question_text="How many times do you eat dessert each week? ({{ scenario.context }})",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7],
)

survey = Survey([q_exercise, q_dessert])

In [14]:
scenarios = ScenarioList.from_list("context", survey_contexts)

In [15]:
agents = AgentList(
    Agent(traits={"age": a}) for a in respondent_ages
)

In [16]:
results = survey.by(scenarios).by(agents).by(m).run()

Job UUID,debdf2b8-d13b-4793-9c22-735be311c329
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/debdf2b8-d13b-4793-9c22-735be311c329
Exceptions Report URL,None
Results UUID,42d25504-d38e-44f3-b0ab-6d00dc10f391
Results URL,https://www.expectedparrot.com/content/42d25504-d38e-44f3-b0ab-6d00dc10f391


In [17]:
results.select("age", "context", "exercise", "dessert")

,agent.age,scenario.context,answer.exercise,answer.dessert
0,You are a teenager (13-19 years old).,nan,3,3
1,You are a teenager (13-19 years old).,You are answering an online survey.,3,3
2,You are a teenager (13-19 years old).,You are being interviewed by a researcher.,4,3
3,You are a teenager (13-19 years old).,You are participating in a focus group of peers.,3,3
4,You are a teenager (13-19 years old).,You are participating in a focus group of people of all ages and backgrounds.,3,3
5,You are college age (20-24 years old).,nan,4,3
6,You are college age (20-24 years old).,You are answering an online survey.,3,3
7,You are college age (20-24 years old).,You are being interviewed by a researcher.,4,3
8,You are college age (20-24 years old).,You are participating in a focus group of peers.,3,3
9,You are college age (20-24 years old).,You are participating in a focus group of people of all ages and backgrounds.,4,3


## Compare prompts
We can compare the prompts that we used which are accessible as fields of the results:

In [18]:
results.columns

,0
0,agent.age
1,agent.agent_index
2,agent.agent_instruction
3,agent.agent_name
4,answer.dessert
5,answer.exercise
6,cache_keys.dessert_cache_key
7,cache_keys.exercise_cache_key
8,cache_used.dessert_cache_used
9,cache_used.exercise_cache_used


In [19]:
results.select("prompt.*")

,prompt.exercise_system_prompt,prompt.exercise_user_prompt,prompt.dessert_system_prompt,prompt.dessert_user_prompt
0,You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a teenager (13-19 years old).'},"How many times do you exercise each week? () 0 : 1 : 2 : 3 : 4 : 5 : 6 : 7 : Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line.",You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a teenager (13-19 years old).'},"How many times do you eat dessert each week? () 0 : 1 : 2 : 3 : 4 : 5 : 6 : 7 : Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line."
1,You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a teenager (13-19 years old).'},"How many times do you exercise each week? (You are answering an online survey.) 0 : 1 : 2 : 3 : 4 : 5 : 6 : 7 : Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line.",You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a teenager (13-19 years old).'},"How many times do you eat dessert each week? (You are answering an online survey.) 0 : 1 : 2 : 3 : 4 : 5 : 6 : 7 : Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line."
2,You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a teenager (13-19 years old).'},"How many times do you exercise each week? (You are being interviewed by a researcher.) 0 : 1 : 2 : 3 : 4 : 5 : 6 : 7 : Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line.",You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a teenager (13-19 years old).'},"How many times do you eat dessert each week? (You are being interviewed by a researcher.) 0 : 1 : 2 : 3 : 4 : 5 : 6 : 7 : Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line."
3,You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a teenager (13-19 years old).'},"How many times do you exercise each week? (You are participating in a focus group of peers.) 0 : 1 : 2 : 3 : 4 : 5 : 6 : 7 : Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line.",You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a teenager (13-19 years old).'},"How many times do you eat dessert each week? (You are participating in a focus group of peers.) 0 : 1 : 2 : 3 : 4 : 5 : 6 : 7 : Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line."
4,You are answering questions as if you were a human. Do not break character.Your traits: {'age': 'You are a te